In [ ]:
%matplotlib inline
#DATASET
import numpy
import pandas
import cv2
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt
import os
import random
import math
import sys
import itertools
import shutil
import glob
from distutils.dir_util import copy_tree
import copy
import zipfile
#TORCH
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
from torchvision import models
from torchvision import transforms
from torchvision import datasets
from torchvision.io import read_image
#METRICS
from tqdm import tqdm
from tqdm import trange
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix
import time

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

dir_name = './Dataset'
if os.path.exists(dir_name) != True:
    
    os.makedirs(dir_name)
    
    with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as z1:
        z1.extractall(dir_name)

    with zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as z2:
        z2.extractall(dir_name)
        
    del z1, z2
    print('Done')
    
train_path = list()
train_label = list()
test_path = list()
test_label = list()
test_id = list()

for var in os.listdir(dir_name + '/train'):
    train_path.append(os.path.join(dir_name, 'train', var))
    
    if 'dog' in var:
        train_label.append(1)
    
    else:
        train_label.append(0)

for var in os.listdir(dir_name + '/test'):
    test_path.append(os.path.join(dir_name, 'test', var))
    test_label.append(0.5)
    test_id.append(int(var[:var.find('.')]))

train_csv = pandas.DataFrame({'id': train_path, 'label': train_label})
test_csv = pandas.DataFrame({'id' : test_path, 'label' : test_label, 'real_id' : test_id})
test_csv = test_csv.sort_values(by=['real_id'])
submission_csv = pandas.read_csv('../input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')

del train_path, train_label, test_path, test_label

device1 = 'cpu'
device2 = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class CustomImageDataset(Dataset):
    
    def __init__(self, img_dir, dir_file, transform=None, target_transform=None):
        self.img_labels = dir_file
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir.iloc[idx]
        
        image = read_image(img_path)
        
        image = image / image.max()
        image = image.type(torch.float64)
        
        if self.transform: 
            image = self.transform(image)
            
        label = self.img_labels.iloc[idx]
        label = torch.tensor(label)
        
        if self.target_transform: 
            label = self.target_transform(label)
        
        return image, label

In [ ]:
transform =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = CustomImageDataset(
    train_csv.id,
    train_csv.label,
    transform = transform, 
    target_transform = None
)

train_dataloader = DataLoader(training_data, batch_size=100, shuffle=True)

test_data = CustomImageDataset(
    test_csv.id,
    test_csv.label,
    transform = transform, 
    target_transform = None
)

test_dataloader = DataLoader(test_data, batch_size=100, shuffle=False)

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)
clear_output(True)

model.load_state_dict(torch.load('../input/classification-model-learning/classifier_model.pt', map_location=torch.device(device2)))
model.eval()
model = model.to(device2)
clear_output()

In [ ]:
def test_model(model, f_n  : str = 'prediction1'):
    
    global device2, submission_csv
    since = time.time()
    pred = list()
    sample_sub = submission_csv
    
    #set model to evaluate mode
    model.eval()
    
    for inputs, _ in tqdm(test_dataloader):
        
        inputs = inputs.type(model.conv1.weight.dtype)
        inputs = inputs.to(device2)
        
        with torch.no_grad():
            
            #forward pass
            output = model(inputs)
            #_, preds = torch.max(output, 1)
            preds = output[:, 1]
            
            for pr in preds:
                pred.append(pr)
    
    
    del pr, preds, output, inputs;
    
    pred = torch.tensor(pred, dtype = torch.int32, device = 'cpu')
    
    assert pred.shape[0] == len(sample_sub)
    
    sample_sub.label = pred
    
    sample_sub.to_csv('{}.csv'.format(f_n), index=False)
    
    return pandas.read_csv('./{}.csv'.format(f_n))

In [ ]:
csv = test_model(model)

In [ ]:
csv